Original Code (as of 5/13/23)

In [56]:
#Import the needed modules
import klayout.db as db
import pandas as pd
from tqdm import tqdm

#Define the function
def rpdb(gds_file:str = 'Fighting_polygon_array.gds', mesh:str = 'fibo_mesh.csv', target_layer:int = 0, LL:list=(0,0), UR:list=(382, 490),output_file:str = 'Test RDP.gds'):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased
    #LL is the lower left corner of the layer to be edited that contains structures, UR is the upper right corner (in um)
    #Will hardcode these for now for testing purposes

    #Import the .gds
    gds = db.Layout()
    gds.read(gds_file)
    print('File read...')

    #Sets the database units as 1nm
    gds.dbu = 0.001

    #Hard coding layer levels (this can be improved in the future...)
    bias_lyr_index = 42
    final_lyr_index = 43

    #Create bias layer for eventual use
    bias_lyr = gds.layer(bias_lyr_index,0,'Initial Biasing Layer')

    #Create final bias layer
    final_lyr = gds.layer(final_lyr_index,0,'Final Biased Layer')

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    print('Mesh read...')

    #Determine mesh columns
    col_len = len(pdb.columns)+1
    columns = range(1,col_len)
    print(f'There are {col_len} columns in the mesh')

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)
    print(f'There are {row_len} rows in the mesh')

    #Determine layer sizing
    x_size = UR[0] - LL[0]
    y_size = UR[1] - LL[1]

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))
    print(f'Mesh X is {mesh_x}nm')
    print(f'Mesh Y is {mesh_y}nm')

    #Calls the top cell (assumes(!) there is one top cell and it contains all the constituent cells)
    top_cell = gds.top_cell()
    print('Top cell chosen, entering loop...')

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in tqdm(rows):
        for j in tqdm(columns):
                #Define the edges of the mesh at each location
                l_edge = LL[0]+mesh_x*(j-1)
                b_edge = UR[1]-mesh_y*i
                r_edge = LL[0]+mesh_x*j
                t_edge = UR[1]-mesh_y*(i-1)

                #Define the biasing at each location
                new_j = str(j)
                biasing = pdb[new_j][i]

                #Takes polygons from the copy layer in a given search region to keep in the "touching" pya.Region 
                search_region = db.DBox(l_edge,b_edge,r_edge,t_edge)
                touching = db.Region(top_cell.begin_shapes_rec_touching(target_layer, search_region))

                #Biases "touching" and copies it into the top cell in the bias layer
                touch_up = touching.sized(int(biasing))
                top_cell.shapes(bias_lyr).insert(touch_up)
                
    
    print("Finished Mesh Loop")

    #Merges any overlapping polygons and places the final biased data onto the "final" layer
    bias_merge = db.Region(top_cell.begin_shapes_rec_touching(bias_lyr,db.DBox(LL[0],LL[1],UR[0],UR[1])))
    bias_merge.merge()
    top_cell.shapes(final_lyr).insert(bias_merge)

    #Removes unneeded bias layer from the layout
    gds.clear_layer(bias_lyr)
    

    #Writes the new file
    return gds.write(output_file)




rpdb()

File read...
Mesh read...
There are 9 columns in the mesh
There are 9 rows in the mesh
Mesh X is 47.75nm
Mesh Y is 61.25nm
Top cell chosen, entering loop...


100%|██████████| 8/8 [00:06<00:00,  1.15it/s]

Finished Mesh Loop


In [59]:
top_cell.keys()

NameError: name 'top_cell' is not defined

New Test (w/ clips) (updated 5/13/23)

In [88]:
#Import the needed modules
import klayout.db as db
import pandas as pd
from tqdm import tqdm

#Define the function
def rpdb(gds_file:str = 'Fighting_polygon_array.gds', mesh:str = 'fibo_mesh.csv', target_layer:int = 0, LL:list=(0,0), UR:list=(99500, 99938),output_file:str = 'Test RDP.gds'):
    #gds is the input gds, and mesh is the matrix of data biasing to be applied, target_layer is the specific layer to be biased
    #LL is the lower left corner of the layer to be edited that contains structures, UR is the upper right corner (in um)
    #Will hardcode these for now for testing purposes

    #Actual extents for test: 99950, 99938

    #Import the .gds
    gds = db.Layout()
    gds.read(gds_file)
    print('File read...')

    #Sets the database units as 1nm
    gds.dbu = 0.001

    #Hard coding layer levels (this can be improved in the future...)
    bias_lyr_index = 49
    
    #Create bias layer for eventual use
    bias_lyr = gds.layer(bias_lyr_index,0,'Initial Biasing Layer')

    #Import the mesh
    pdb = pd.read_csv(mesh, index_col=0)
    print('Mesh read...')

    #Determine maximum absolute bias applied, in order to size up the cell regions
    mesh_max = pdb.max().max()
    mesh_min = pdb.min().min()
    if abs(mesh_max) >= abs(mesh_min):
         bias_max = mesh_max
    else:
         bias_max = abs(mesh_min)

    #Determine mesh columns
    col_len = len(pdb.columns)+1
    columns = range(1,col_len)
    print(f'There are {col_len} columns in the mesh')

    #Determine mesh rows
    row_len = len(pdb)+1
    rows = range(1,row_len)
    print(f'There are {row_len} rows in the mesh')

    #Determine layer sizing
    x_size = UR[0] - LL[0]
    y_size = UR[1] - LL[1]

    #Define size of meshing
    mesh_x = x_size / (len(columns))
    mesh_y = y_size / (len(rows))
    print(f'Mesh X is {mesh_x}nm')
    print(f'Mesh Y is {mesh_y}nm')

    #Calls the top cell (assumes(!) there is one top cell and it contains all the constituent cells) and stores index of all called cells
    top_cell = gds.top_cell()
    top_cell_index = top_cell.cell_index()
    called_cells = top_cell.called_cells()
    print(f'The called cell indicies are: {called_cells}')
    print('Top cell chosen, entering loop...')

    #Loop across the mesh, define the regions in the copy layer to copy into the bias layer, then resize the features in the bias layer based on the mesh input
    for i in tqdm(rows):
        for j in columns:
                #Define the edges of the mesh at each location
                l_edge = LL[0]+(mesh_x*(j-1)-bias_max)
                b_edge = UR[1]-(mesh_y*i+bias_max)
                r_edge = LL[0]+(mesh_x*j+bias_max)
                t_edge = UR[1]-(mesh_y*(i-1)-bias_max)

                #Define the biasing at each location
                new_j = str(j)
                biasing = pdb[new_j][i]
                #print(f'This is the bias value: {biasing}')

                #Clips mesh area from the polygon, preserving hierarchy, and names it
                clip = gds.clip(top_cell,db.DBox(l_edge,b_edge,r_edge,t_edge))
                clip.name= f'Mesh_Cell_{i}_{j}'
                

                #Loop to identify each shape and size it up by the bias value amount
                #print(f'Clip called cells are: {clip.called_cells()}')

                #Copies the cell tree from the clip into a new bias cell
                bias_cell = gds.create_cell(f'bias_cell_{i}_{j}')
                bias_tree = bias_cell.copy_tree(clip)
                print(f'Bias cell tree indicies are {bias_cell.called_cells()}')
                print(f'Biasing is {biasing}')

                #Goes through the hierarchy of each clip and biases by the appropriate value, and inserts into the bias layer
                for k in bias_cell.called_cells():
                    copy_cell = gds.cell(k)
                    search_region = db.Region(copy_cell.bbox(target_layer))
                    shapes_region = db.Region(copy_cell.shapes(target_layer))
                    touching = search_region & shapes_region
                    bias_apply = touching.sized(int(biasing))
                    copy_cell.shapes(bias_lyr).insert(bias_apply)

                                    
                clip.prune_cell()                    
    
    print("Finished Mesh Loop")
    
    #Writes the new file
    return gds.write(output_file)

rpdb()

File read...
Mesh read...
There are 9 columns in the mesh
There are 9 rows in the mesh
Mesh X is 12437.5nm
Mesh Y is 12492.25nm
The called cell indicies are: [0, 1, 2, 3]
Top cell chosen, entering loop...


  0%|          | 0/8 [00:00<?, ?it/s]

Bias cell tree indicies are [10, 11, 12, 13, 14, 15, 16]
Biasing is 1
Bias cell tree indicies are [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
Biasing is 1
Bias cell tree indicies are [44, 45, 46, 47, 48, 49, 50, 51, 52]
Biasing is 2
Bias cell tree indicies are [58, 59, 60, 61, 62, 63, 64]
Biasing is 3
Bias cell tree indicies are [81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]
Biasing is 5
Bias cell tree indicies are [108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118]
Biasing is 8


 12%|█▎        | 1/8 [00:00<00:02,  2.49it/s]

Bias cell tree indicies are [130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142]
Biasing is 13
Bias cell tree indicies are [161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180]
Biasing is 21
Bias cell tree indicies are [185, 186, 187, 188, 189, 190]
Biasing is 1
Bias cell tree indicies are [199, 200, 201, 202, 203, 204, 205, 206, 207, 208]
Biasing is -1
Bias cell tree indicies are [215, 216, 217, 218, 219, 220, 221, 222]
Biasing is 2
Bias cell tree indicies are [227, 228, 229, 230, 231, 232]
Biasing is 3
Bias cell tree indicies are [247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262]
Biasing is 5
Bias cell tree indicies are [271, 272, 273, 274, 275, 276, 277, 278, 279, 280]
Biasing is 8
Bias cell tree indicies are [290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300]
Biasing is 13


...

Bias cell tree indicies are [1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044]
Biasing is 21
Bias cell tree indicies are [1050, 1051, 1052, 1053, 1054, 1055, 1056]
Biasing is 13
Bias cell tree indicies are [1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076]
Biasing is -3
Bias cell tree indicies are [1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092]
Biasing is -4
Bias cell tree indicies are [1098, 1099, 1100, 1101, 1102, 1103, 1104]
Biasing is 3
Bias cell tree indicies are [1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136]
Biasing is 5
Bias cell tree indicies are [1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156]
Biasing is 32


 88%|████████▊ | 7/8 [00:10<00:02,  2.01s/it]

Bias cell tree indicies are [1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178]
Biasing is -13
Bias cell tree indicies are [1195, 1196, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212]
Biasing is 21
Bias cell tree indicies are [1218, 1219, 1220, 1221, 1222, 1223, 1224]
Biasing is 21
Bias cell tree indicies are [1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244]
Biasing is -5
Bias cell tree indicies are [1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260]
Biasing is -6
Bias cell tree indicies are [1266, 1267, 1268, 1269, 1270, 1271, 1272]
Biasing is 6
Bias cell tree indicies are [1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306]
Biasing is 15
Bias cell tree indicies are [1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326]
Biasing is 24
Bias cell tree indicies are [1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 134

100%|██████████| 8/8 [00:13<00:00,  1.73s/it]

Bias cell tree indicies are [1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388]
Biasing is -21
Finished Mesh Loop
